In [1]:
import pandas as pd
import numpy as np
import os
import sys
from collections import defaultdict, Counter

input_data_folder = "../logdata/orig/"
output_data_folder = "../logdata/"
in_filename = "Invoice Approval.csv"

In [2]:
case_id_col = "Case ID"
activity_col = "Activity"
timestamp_col = "Complete Timestamp"

In [3]:
category_freq_threshold = 10

In [4]:
# features for classifier
dynamic_cat_cols = ["Activity", 'Resource', 'ActivityFinalAction', "EventType"] 
static_cat_cols = ["CostCenter.Code", "Supplier.City", "Supplier.Name", "Supplier.State"]
dynamic_num_cols = []
static_num_cols = ["InvoiceTotalAmountWithoutVAT"]

In [5]:
static_cols = static_cat_cols + static_num_cols + [case_id_col]
dynamic_cols = dynamic_cat_cols + dynamic_num_cols + [timestamp_col]
cat_cols = dynamic_cat_cols + static_cat_cols

In [6]:
def extract_timestamp_features(group):
    
    group = group.sort_values(timestamp_col, ascending=False, kind='mergesort')
    
    tmp = group[timestamp_col] - group[timestamp_col].shift(-1)
    tmp = tmp.fillna(0)
    group["timesincelastevent"] = tmp.apply(lambda x: float(x / np.timedelta64(1, 's'))) # m is for minutes

    tmp = group[timestamp_col] - group[timestamp_col].iloc[-1]
    tmp = tmp.fillna(0)
    group["timesincecasestart"] = tmp.apply(lambda x: float(x / np.timedelta64(1, 's'))) # m is for minutes
    
    tmp = group[timestamp_col].iloc[0] - group[timestamp_col]
    tmp = tmp.fillna(0)
    group["remtime"] = tmp.apply(lambda x: float(x / np.timedelta64(1, 's')))

    group = group.sort_values(timestamp_col, ascending=True, kind='mergesort')
    group["event_nr"] = range(1, len(group) + 1)
    
    return group

def check_if_any_of_activities_exist(group, activities):
    if np.sum(group[activity_col].isin(activities)) > 0:
        return True
    else:
        return False    


In [7]:
data = pd.read_csv(os.path.join(input_data_folder, in_filename), sep=",")
data.rename(columns=lambda x: x.replace('(case) ', ''), inplace=True)

In [14]:
data.head(12)

CostCenter.Code Supplier.City            Supplier.Name  \
Case ID                                                             
977017  0      CostCenter1      Abeokuta      Enim Nec Tempus LLP   
        1      CostCenter1      Abeokuta      Enim Nec Tempus LLP   
        2      CostCenter1      Abeokuta      Enim Nec Tempus LLP   
        3      CostCenter1      Abeokuta      Enim Nec Tempus LLP   
        4      CostCenter1      Abeokuta      Enim Nec Tempus LLP   
        5      CostCenter1      Abeokuta      Enim Nec Tempus LLP   
        6      CostCenter1      Abeokuta      Enim Nec Tempus LLP   
        7      CostCenter1      Abeokuta      Enim Nec Tempus LLP   
977022  11     CostCenter2  Pont-de-Loup  Augue Porttitor Limited   
        12     CostCenter2  Pont-de-Loup  Augue Porttitor Limited   
        13     CostCenter2  Pont-de-Loup  Augue Porttitor Limited   
        14     CostCenter2  Pont-de-Loup  Augue Porttitor Limited   

           Supplier.State  InvoiceTotalAmountWithoutVAT  Case ID  \
Case ID                                                            
977017  0         Algeria                          4145   977017   
        1         Algeria                          4145   977017   
        2         Algeria                          4145   977017   
        3         Algeria                          4145   977017   
        4         Algeria                          4145   977017   
        5         Algeria                          4145   977017   
        6         Algeria                          4145   977017   
        7         Algeria                          4145   977017   
977022  11         Jordan                          2826   977022   
        12         Jordan                          2826   977022   
        13         Jordan                          2826   977022   
        14         Jordan                          2826   977022   

                                   Activity     Resource ActivityFinalAction  \
Case ID                                                                        
977017  0                             Start       Server                 NaN   
        1                     Process start       Server                 NaN   
        2   Status change to Being Approved       Server                 NaN   
        3               Check order numbers       Server                 NaN   
        4                   Compare of sums       Server                 NaN   
        5                 Check cost center       Server                 NaN   
        6         Get lowest approval level       Server                 NaN   
        7       Approving on specific level  Lara Obrien            Approved   
977022  11                            Start       Server                 NaN   
        12                    Process start       Server                 NaN   
        13  Status change to Being Approved       Server                 NaN   
        14              Check order numbers       Server                 NaN   

            EventType  Complete Timestamp  month  weekday  hour  \
Case ID                                                           
977017  0           0 2015-02-18 14:38:03      2        2    14   
        1           1 2015-02-18 14:38:06      2        2    14   
        2           1 2015-02-18 14:38:12      2        2    14   
        3           1 2015-02-18 14:38:12      2        2    14   
        4           1 2015-02-18 14:38:13      2        2    14   
        5           1 2015-02-18 14:38:13      2        2    14   
        6           1 2015-02-18 14:38:14      2        2    14   
        7           2 2015-02-18 14:38:17      2        2    14   
977022  11          0 2015-02-18 14:39:42      2        2    14   
        12          1 2015-02-18 14:39:43      2        2    14   
        13          1 2015-02-18 14:39:43      2        2    14   
        14          1 2015-02-18 14:39:44      2        2    14   

            timesincelastevent 

In [30]:
data.head(12)

CostCenter.Code Supplier.City            Supplier.Name  \
Case ID                                                             
977017  0      CostCenter1      Abeokuta      Enim Nec Tempus LLP   
        1      CostCenter1      Abeokuta      Enim Nec Tempus LLP   
        2      CostCenter1      Abeokuta      Enim Nec Tempus LLP   
        3      CostCenter1      Abeokuta      Enim Nec Tempus LLP   
        4      CostCenter1      Abeokuta      Enim Nec Tempus LLP   
        5      CostCenter1      Abeokuta      Enim Nec Tempus LLP   
        6      CostCenter1      Abeokuta      Enim Nec Tempus LLP   
        7      CostCenter1      Abeokuta      Enim Nec Tempus LLP   
977022  11     CostCenter2  Pont-de-Loup  Augue Porttitor Limited   
        12     CostCenter2  Pont-de-Loup  Augue Porttitor Limited   
        13     CostCenter2  Pont-de-Loup  Augue Porttitor Limited   
        14     CostCenter2  Pont-de-Loup  Augue Porttitor Limited   

           Supplier.State  InvoiceTotalAmountWithoutVAT  Case ID  \
Case ID                                                            
977017  0         Algeria                          4145   977017   
        1         Algeria                          4145   977017   
        2         Algeria                          4145   977017   
        3         Algeria                          4145   977017   
        4         Algeria                          4145   977017   
        5         Algeria                          4145   977017   
        6         Algeria                          4145   977017   
        7         Algeria                          4145   977017   
977022  11         Jordan                          2826   977022   
        12         Jordan                          2826   977022   
        13         Jordan                          2826   977022   
        14         Jordan                          2826   977022   

                                   Activity     Resource ActivityFinalAction  \
Case ID                                                                        
977017  0                             Start       Server             missing   
        1                     Process start       Server             missing   
        2   Status change to Being Approved       Server             missing   
        3               Check order numbers       Server             missing   
        4                   Compare of sums       Server             missing   
        5                 Check cost center       Server             missing   
        6         Get lowest approval level       Server             missing   
        7       Approving on specific level  Lara Obrien            Approved   
977022  11                            Start       Server             missing   
        12                    Process start       Server             missing   
        13  Status change to Being Approved       Server             missing   
        14              Check order numbers       Server             missing   

            EventType  Complete Timestamp  month  weekday  hour  \
Case ID                                                           
977017  0           0 2015-02-18 14:38:03      2        2    14   
        1           1 2015-02-18 14:38:06      2        2    14   
        2           1 2015-02-18 14:38:12      2        2    14   
        3           1 2015-02-18 14:38:12      2        2    14   
        4           1 2015-02-18 14:38:13      2        2    14   
        5           1 2015-02-18 14:38:13      2        2    14   
        6           1 2015-02-18 14:38:14      2        2    14   
        7           2 2015-02-18 14:38:17      2        2    14   
977022  11          0 2015-02-18 14:39:42      2        2    14   
        12          1 2015-02-18 14:39:43      2        2    14   
        13          1 2015-02-18 14:39:43      2        2    14   
        14          1 2015-02-18 14:39:44      2        2    14   

            timesincelastevent 

In [9]:
# discard incomplete cases
last_events = data.sort_values([timestamp_col], ascending=True, kind='mergesort').groupby(case_id_col).last()["Activity"]
complete_cases = last_events.index[last_events=="Process end"]
data = data[data[case_id_col].isin(complete_cases)]

In [11]:
data = data[static_cols + dynamic_cols]

# add features extracted from timestamp
data[timestamp_col] = pd.to_datetime(data[timestamp_col])
data["month"] = data[timestamp_col].dt.month
data["weekday"] = data[timestamp_col].dt.weekday
data["hour"] = data[timestamp_col].dt.hour
data = data.groupby(case_id_col).apply(extract_timestamp_features)

In [13]:
# add inter-case features
print("Extracting open cases...")
sys.stdout.flush()
data = data.sort_values([timestamp_col], ascending=True, kind='mergesort')
dt_first_last_timestamps = data.groupby(case_id_col)[timestamp_col].agg([min, max])
dt_first_last_timestamps.columns = ["start_time", "end_time"]
#data["open_cases"] = data[timestamp_col].apply(get_open_cases)
case_end_times = dt_first_last_timestamps.to_dict()["end_time"]

data["open_cases"] = 0
case_dict_state = {}
for idx, row in data.iterrows():
    case = row[case_id_col]
    current_ts = row[timestamp_col]

    # save the state
    data.set_value(idx, 'open_cases', len(case_dict_state))

    if current_ts >= case_end_times[case]:
        if case in case_dict_state:
            del case_dict_state[case]
    else:
        case_dict_state[case] = 1


Extracting open cases...


/home/coderus/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: FutureWarning: 'Case ID' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  """
/home/coderus/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:17: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [15]:
def get_open_cases(date):
    return sum((dt_first_last_timestamps["start_time"] <= date) & (dt_first_last_timestamps["end_time"] > date))
# add inter-case features - 2nd way
data = data.sort_values([timestamp_col], ascending=True, kind='mergesort')
dt_first_last_timestamps = data.groupby(case_id_col)[timestamp_col].agg([min, max])
dt_first_last_timestamps.columns = ["start_time", "end_time"]
data["open_cases2"] = data[timestamp_col].apply(get_open_cases)

/home/coderus/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: FutureWarning: 'Case ID' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  """


In [20]:
data["open_cases3"] = data["open_cases2"] - data["open_cases"]

In [28]:
data.columns

Index([u'CostCenter.Code', u'Supplier.City', u'Supplier.Name',
       u'Supplier.State', u'InvoiceTotalAmountWithoutVAT', u'Case ID',
       u'Activity', u'Resource', u'ActivityFinalAction', u'EventType',
       u'Complete Timestamp', u'month', u'weekday', u'hour',
       u'timesincelastevent', u'timesincecasestart', u'remtime', u'event_nr',
       u'open_cases'],
      dtype='object')

In [29]:
print("Imputing missing values...")
sys.stdout.flush()
# impute missing values
grouped = data.sort_values(timestamp_col, ascending=True, kind='mergesort').groupby(case_id_col)
for col in static_cols + dynamic_cols:
    data[col] = grouped[col].transform(lambda grp: grp.fillna(method='ffill'))
        
data[cat_cols] = data[cat_cols].fillna('missing')
data = data.fillna(0)

Imputing missing values...


/home/coderus/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: 'Case ID' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  after removing the cwd from sys.path.


In [27]:
data = data.drop("open_cases3", axis =1)

In [31]:
# set infrequent factor levels to "other"
for col in cat_cols:
    counts = data[col].value_counts()
    mask = data[col].isin(counts[counts >= category_freq_threshold].index)
    data.loc[~mask, col] = "other"
    
data = data.sort_values(timestamp_col, ascending=True, kind="mergesort")    

In [32]:
data.to_csv(os.path.join(output_data_folder, "minit_invoice.csv"), sep=";", index=False)

In [ ]:
data = pd.read_csv(os.path.join(input_data_folder, in_filename), sep=";")

In [ ]:
data.head()

In [ ]:
def get_case_length(group):
    group['case_length'] = group.shape[0]
    return group

In [ ]:
print(data.shape)

In [ ]:
data = data.groupby(case_id_col).apply(get_case_length)

In [ ]:
data = data[data["case_length"] > 2]

In [ ]:
data.groupby("case_length")[case_id_col].nunique()

In [ ]:
data.shape

In [ ]:
data2.shape

In [ ]:
data2 = data[data["case_length"] > 2]

In [ ]:
data2 = data2.drop("case_length", axis=1)

In [ ]:
data2.to_csv(os.path.join(output_data_folder, in_filename), sep=";", index=False)

In [ ]:
(428626 - 411978)/2

In [ ]:
data2.groupby("case_length")[case_id_col].nunique()

In [ ]:
data2.shape

In [ ]:
data2[case_id_col].nunique()